In [1]:
# import stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('19981201 Open Electricity.csv')

In [2]:
data.head(5)
#data.info()

,date,Battery (Charging) - GWh,Pumps - GWh,Coal (Brown) - GWh,Coal (Black) - GWh,Bioenergy (Biomass) - GWh,Distillate - GWh,Gas (Steam) - GWh,Gas (CCGT) - GWh,Gas (OCGT) - GWh,...,Gas (Steam) Market Value - AUD,Gas (CCGT) Market Value - AUD,Gas (OCGT) Market Value - AUD,Gas (Reciprocating) Market Value - AUD,Gas (Waste Coal Mine) Market Value - AUD,Battery (Discharging) Market Value - AUD,Hydro Market Value - AUD,Wind Market Value - AUD,Solar (Utility) Market Value - AUD,Solar (Rooftop) Market Value - AUD
0,1998-12-01,0.0,-24.36,3277.81,6472.65,0.0,15.22,233.59,171.45,71.79,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1999-01-01,0.0,-42.41,4351.95,8146.82,0.0,3.58,349.21,216.67,108.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1999-02-01,0.0,-21.76,4153.15,7545.69,0.0,2.11,349.69,203.70,110.14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1999-03-01,0.0,-64.14,4622.72,8217.25,0.0,11.16,253.31,235.47,124.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1999-04-01,0.0,-18.47,4204.93,7707.56,0.0,1.74,350.19,221.98,87.84,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# clean up the data
# all the columns are numeric, only datetime needs fixing
data['date'] = pd.to_datetime(data['date'])

# set the date as index
data.set_index('date', inplace=True)

In [4]:
for col in data.columns:
    print(col)

Battery (Charging) -  GWh
Pumps -  GWh
Coal (Brown) -  GWh
Coal (Black) -  GWh
Bioenergy (Biomass) -  GWh
Distillate -  GWh
Gas (Steam) -  GWh
Gas (CCGT) -  GWh
Gas (OCGT) -  GWh
Gas (Reciprocating) -  GWh
Gas (Waste Coal Mine) -  GWh
Battery (Discharging) -  GWh
Hydro -  GWh
Wind -  GWh
Solar (Utility) -  GWh
Solar (Rooftop) -  GWh
Coal (Brown) Emissions Vol - tCO₂e
Coal (Black) Emissions Vol - tCO₂e
Bioenergy (Biomass) Emissions Vol - tCO₂e
Distillate Emissions Vol - tCO₂e
Gas (Steam) Emissions Vol - tCO₂e
Gas (CCGT) Emissions Vol - tCO₂e
Gas (OCGT) Emissions Vol - tCO₂e
Gas (Reciprocating) Emissions Vol - tCO₂e
Gas (Waste Coal Mine) Emissions Vol - tCO₂e
Emissions Intensity - kgCO₂e/MWh
Volume Weighted Price - AUD/MWh
Battery (Charging) Market Value - AUD
Pumps Market Value - AUD
Coal (Brown) Market Value - AUD
Coal (Black) Market Value - AUD
Bioenergy (Biomass) Market Value - AUD
Distillate Market Value - AUD
Gas (Steam) Market Value - AUD
Gas (CCGT) Market Value - AUD
Gas (OCGT) M